In [32]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from IPython.display import display
from IPython.display import set_matplotlib_formats
import datetime
import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.cluster import KMeans
import os
import warnings
import gc
import copy
set_matplotlib_formats('svg')

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 200)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
%%time
# ['销售订单编号', '物料编号', '是否三包描述', '基本计量单位', '产品组名称', '产品组代码', 
# '创建日期', 'year', 'month', 'week', 'day', 'weekday', '订单数量', '订单金额', '物料描述', 
# '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况', '吨位区间', '设备类型']
df = pd.read_csv('gongqi.csv', encoding='utf-8-sig',
                 usecols=['销售订单编号', '物料编号', '是否三包描述', '基本计量单位', 
                          '创建日期', 'year', 'month', 'day', 
                          '订单数量', '订单金额', 
                          '物料描述', '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况', '吨位区间', '设备类型'])
print('物料编号数量：', len(df['物料编号'].unique()))
print('订单属性包括：', df.columns)

# 存在订单金额为0的订单，三包单或者赠送订单，不好处理
# 物料类别有冲突这里只取一种类别，后续让业务解决冲突
# 先不管这些问题，不做筛选，一起来整体看看

# 考虑去掉订单金额为0的订单，赠送订单
print('订单金额为0的疑似赠送订单数：', len(df.loc[df['订单金额']==0]))
# df = df.loc[df['订单金额']!=0]

# 考虑三包件
display(df['是否三包描述'].value_counts())
print(len(df.loc[df['是否三包描述']=='三包申请单', '物料编号'].unique()))
print(len(df.loc[df['是否三包描述']=='销售申请', '物料编号'].unique()))

# 根据订单金额和数量求出物料单价，以此作为成本
df['物料单价'] = df['订单金额'] / df['订单数量']

# 部分物料类别合并
# 不确定合并的对不对，另外还有诸如吊臂、支腿、两室、等不确定要不要合并
df.loc[df['物料类别']=='液压件', '物料类别'] = '液压件类'
df.loc[df['物料类别']=='大型结构件', '物料类别'] = '大型结构件类'
df.loc[df['物料类别']=='小型结构件(含非锻件五金件)', '物料类别'] = '小型结构件'
df.loc[df['物料类别']=='弹簧类', '物料类别'] = '弹簧件类'

物料编号数量： 20311
订单属性包括： Index(['销售订单编号', '物料编号', '是否三包描述', '基本计量单位', '创建日期', 'year', 'month', 'day',
       '订单数量', '订单金额', '物料描述', '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况',
       '吨位区间', '设备类型'],
      dtype='object')
订单金额为0的疑似赠送订单数： 52527


销售申请     256221
三包申请单     18616
Name: 是否三包描述, dtype: int64

4376
19337
Wall time: 1.66 s


In [3]:
# 构造包含完整月份的订单数据，没有的月份订单数量补0
# 无物料类别信息等业务信息的，补未知
def struct_full_month_df(df):
    full_month_df = pd.DataFrame(product(df['物料编号'].unique(), 
                         np.sort(df['year'].unique()),
                         np.sort(df['month'].unique())),
                columns=['物料编号', 'year', 'month'])
    temp1 = df.groupby(['物料编号', 'year', 'month'], as_index=False)['订单数量'].agg({'月订单数量':'sum'})
    temp2 = df.groupby(['物料类别', '物料编号'], as_index=False).count()
    temp3 = df.groupby(['基本计量单位', '物料编号'], as_index=False).count()
    temp4 = df.groupby(['是否进口件', '物料编号'], as_index=False).count()
    temp5 = df.groupby(['周期（天）下计划后的交货周期', '物料编号'], as_index=False).count()
    temp6 = df.groupby(['在产情况', '物料编号'], as_index=False).count()
    temp7 = df.groupby(['物料描述', '物料编号'], as_index=False).count()
    temp8 = df.groupby(['吨位区间', '物料编号'], as_index=False).count()
    temp9 = df.groupby(['设备类型', '物料编号'], as_index=False).count()
    temp10 = df.groupby(['物料编号'], as_index=False)['物料单价'].agg({'物料单位成本':'max'})
    full_month_df = pd.merge(full_month_df, temp1, on=['物料编号', 'year', 'month'], how='left')
    full_month_df = pd.merge(full_month_df, temp2[['物料类别', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp3[['基本计量单位', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp4[['是否进口件', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp5[['周期（天）下计划后的交货周期', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp6[['在产情况', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp7[['物料描述', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp8[['吨位区间', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp9[['设备类型', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp10, on=['物料编号'], how='left')
    full_month_df['月订单数量'].fillna(0, inplace=True)
    full_month_df.fillna('未知', inplace=True)
    return full_month_df

def plot_monthly(data, ylabel='月订单数量', title='各年份各类配件需求总量随月份变化图'):
    
    plt.figure(figsize=(10, 3))
    x = data['date']
    y = data[ylabel]
#     plt.plot(x, y)
    plt.bar(x, y, width=5)
    plt.xticks(fontsize=10)    
    plt.yticks(fontsize=10) 
    plt.xlabel('月份', fontsize=10)    
    plt.ylabel('需求', fontsize=10)
    plt.title(title, fontsize=10)
    plt.grid()
    plt.gcf().autofmt_xdate() # 自动旋转日期标记
    plt.show()

# 思路1：计算需求间隔平均值
# 注意计算间隔的方式，算序列前面的0，不算序列后面的0，只要没到非0值就一直计数，到非0值就重新计数
# 定义是否合理，如果序列后面有很多0呢？？？？后续应该在序列最后补一个非0值来计算间隔，也有问题？？？？
# 平均间隔单独看有各种问题，需要考虑结合0值的比例，平均值等信息
# def interval_mean(input_endog):
    
#     input_series = np.asarray(input_endog)
# #     input_length = len(input_series)
#     nzd = np.where(input_series != 0)[0] # find location of non-zero demand
# #     z = input_series[nzd] # demand
#     x = np.concatenate([[nzd[0]], np.diff(nzd)]) # intervals
#     # input_series = [0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]
#     # nzd = [ 5  6  8 11 16]
#     # x = [5 1 2 3 5]
#     p = np.mean(x)
#     if list(nzd) == []: # 序列里全是0，此时，均值为0，方差为0，间隔为nan
#         p = np.nan
#     if list(nzd) == [0]: # 序列开头非0，后面全是0，此时，间隔给一个大值，20？
#         p = 20
#     # 输出求间隔平均值
#     return p

# 思路2：计算需求间隔平均值
# 考虑几个特殊情况，如果不处理，对平均间隔计算有较大影响
def interval_mean(input_endog):
    
    input_series = np.asarray(input_endog)
    # 考虑序列全0的情况，定义间隔长度为序列长度
    if np.isin(input_series, [0]).all():
        return len(input_series)
    # 考虑序列后面一个或多个0结尾的情况，补一个0来定义间隔长度
    # 补0操作这种合理吗，比如[0,0,0,0,0,0,1,0]和[0,0,0,0,0,0,0,1]的平均间隔就有一定差距
    if input_series[-1]==0:
        input_series = np.append(input_series, 1)
    nzd = np.where(input_series != 0)[0] # find location of non-zero demand
    if nzd[0]!=0:
        x = np.concatenate([[nzd[0]+1], np.diff(nzd)]) # intervals
    else:
        # 考虑序列开头为非0的情况，按照上面的代码会计算出间隔为0，这里针对这种情况特殊处理
        x = np.diff(nzd)
    p = np.mean(x)
    # 输出求间隔平均值
    return p

In [4]:
matrl_full = struct_full_month_df(df)
print(len(matrl_full))
matrl_full.head()

487464


,物料编号,year,month,月订单数量,物料类别,基本计量单位,是否进口件,周期（天）下计划后的交货周期,在产情况,物料描述,吨位区间,设备类型,物料单位成本
0,00630736400001010,2020,1,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
1,00630736400001010,2020,2,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
2,00630736400001010,2020,3,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
3,00630736400001010,2020,4,0.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
4,00630736400001010,2020,5,0.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81


In [5]:
# 计算每个物料的各种统计量，以及保留相关业务属性（物料类别、计量单位等）
matrl_stats = matrl_full.groupby('物料编号', as_index=False)['月订单数量'].agg({'单配件月需求均值':'mean'})
matrl_stats['单配件月需求最大值'] = matrl_full.groupby('物料编号')['月订单数量'].max().values
matrl_stats['单配件月需求标准差'] = matrl_full.groupby('物料编号')['月订单数量'].std().values
matrl_stats['单配件月需求变异系数'] = matrl_stats['单配件月需求标准差'] / matrl_stats['单配件月需求均值']
matrl_stats['需求平均间隔'] = matrl_full.groupby('物料编号')['月订单数量'].apply(interval_mean).values

# 把月订单数量的0值转换为空值，方便统计非0统计量
matrl_full.loc[matrl_full['月订单数量']==0, '月订单数量'] = np.nan
matrl_stats['单配件月非0需求均值'] = matrl_full.groupby('物料编号')['月订单数量'].mean().values
matrl_stats['单配件月非0需求标准差'] = matrl_full.groupby('物料编号')['月订单数量'].std().values
matrl_stats['单配件月非0需求变异系数'] = matrl_stats['单配件月非0需求标准差'] / matrl_stats['单配件月非0需求均值']

matrl_stats['单月0需求比例'] = 1 - matrl_full.groupby('物料编号')['月订单数量'].count().values / 24
# 统计完以后，把月订单数量的空值重新填0
matrl_full['月订单数量'].fillna(0, inplace=True)

# 加上业务信息
id_class = matrl_full.groupby(['物料编号','物料类别'],as_index=False).count()[['物料编号','物料类别']]
matrl_stats = pd.merge(matrl_stats, id_class, on='物料编号', how='left')
id_measuring = matrl_full.groupby(['物料编号','基本计量单位'],as_index=False).count()[['物料编号','基本计量单位']]
matrl_stats = pd.merge(matrl_stats, id_measuring, on='物料编号', how='left')
id_description = matrl_full.groupby(['物料编号','物料描述'],as_index=False).count()[['物料编号','物料描述']]
matrl_stats = pd.merge(matrl_stats, id_description, on='物料编号', how='left')
id_entrance = matrl_full.groupby(['物料编号','是否进口件'],as_index=False).count()[['物料编号','是否进口件']]
matrl_stats = pd.merge(matrl_stats, id_entrance, on='物料编号', how='left')
id_period = matrl_full.groupby(['物料编号','周期（天）下计划后的交货周期'],as_index=False).count()[['物料编号','周期（天）下计划后的交货周期']]
matrl_stats = pd.merge(matrl_stats, id_period, on='物料编号', how='left')
id_in_production = matrl_full.groupby(['物料编号','在产情况'],as_index=False).count()[['物料编号','在产情况']]
matrl_stats = pd.merge(matrl_stats, id_in_production, on='物料编号', how='left')
id_tonnage = matrl_full.groupby(['物料编号','吨位区间'],as_index=False).count()[['物料编号','吨位区间']]
matrl_stats = pd.merge(matrl_stats, id_tonnage, on='物料编号', how='left')
id_type = matrl_full.groupby(['物料编号','设备类型'],as_index=False).count()[['物料编号','设备类型']]
matrl_stats = pd.merge(matrl_stats, id_type, on='物料编号', how='left')
id_money = matrl_full.groupby(['物料编号'],as_index=False)['物料单位成本'].agg({'物料单位成本':'max'})
matrl_stats = pd.merge(matrl_stats, id_money, on='物料编号', how='left')

一、先考虑在产情况，停产的无需考虑，未知先不考虑，只保留在产的配件进行后续分类

In [6]:
matrl_stats['在产情况'].value_counts()

停产    9466
在产    9157
未知    1688
Name: 在产情况, dtype: int64

二、考虑是否进口件，进口件类别较少且交货周期长

In [7]:
display(matrl_stats.loc[matrl_stats['是否进口件']=='是', '物料类别'].value_counts())
display(matrl_stats.loc[matrl_stats['是否进口件']=='是', '周期（天）下计划后的交货周期'].value_counts())

泵和阀类        128
发动机类        119
传动件传动操纵类     79
一般结构件类       61
滤芯类          31
钢丝绳类         27
电器件类         18
底盘车桥类        10
马达类           8
减速机           7
管和接头类         5
油品类           5
液压件类          4
国标件           3
油缸类           1
气动元件类         1
弹簧件类          1
Name: 物料类别, dtype: int64

90.0     497
150.0     11
Name: 周期（天）下计划后的交货周期, dtype: int64

三、考虑计量单位，一些事实上非配件的物料（单位为米、平方米、立方米、毫升、毫米、公斤、升等）可以另外考虑，如油品、图册等

In [41]:
# 不同品类的计量单位
# 需要考虑不同计量单位的影响
display(matrl_stats['基本计量单位'].value_counts())
display(matrl_stats.groupby(['物料类别', '基本计量单位'], as_index=False)['物料编号'].agg({'物料数': 'count'}))

# 部分物料的计量单位可能存在错误，需矫正
# 计量单位存疑物料
# matrl_stats.loc[(matrl_stats['物料类别']=='一般结构件类') & (matrl_stats['基本计量单位']=='KG')]
# matrl_stats.loc[(matrl_stats['物料类别']=='两室配件') & (matrl_stats['基本计量单位']=='KG')]
# matrl_stats.loc[(matrl_stats['物料类别']=='传动件传动操纵类') & (matrl_stats['基本计量单位']=='KG')]
# matrl_stats.loc[(matrl_stats['物料类别']=='发动机类') & (matrl_stats['基本计量单位']=='L')] # 或许应该放到油品类
# matrl_stats.loc[(matrl_stats['物料类别']=='标牌图册类') 
#                 & ((matrl_stats['基本计量单位']=='MM') | (matrl_stats['基本计量单位']=='M'))] # 反光标识膜这种单位很小，导致突然的数值很大
# matrl_stats.loc[(matrl_stats['物料类别']=='泵和阀类') & (matrl_stats['基本计量单位']=='KG')]
# matrl_stats.loc[(matrl_stats['物料类别']=='电器件类') & (matrl_stats['基本计量单位']=='KG')]

PC     19631
EA       550
KG        51
M         49
L         24
TAO        2
MM         2
M3         1
TAI        1
Name: 基本计量单位, dtype: int64

,物料类别,基本计量单位,物料数
0,一般结构件类,EA,1
1,一般结构件类,KG,2
2,一般结构件类,PC,4358
3,两室,PC,105
4,两室配件,EA,1
5,两室配件,KG,7
6,两室配件,PC,646
7,传动件传动操纵类,EA,66
8,传动件传动操纵类,KG,3
9,传动件传动操纵类,PC,773


四、包括新进入市场的配件和退出市场的配件，以及生产期的配件：  
退出市场的配件即停产配件，不需要考虑；  
新进入市场的配件也可能非0需求频次较低，最好还是有业务告知，销售月份较少的时候不预测或简单方法预测，结合量、价值和重要性选择备货策略；  
<big>**这里会引出增量训练的问题，新配件如何处理，新月份数据如何增量训练？**</big>

In [8]:
# 从订单数据的角度考虑新旧配件
new_old_df = copy.deepcopy(matrl_full[['物料编号', 'year', 'month', '月订单数量']])
new_old_df['year_month'] = (new_old_df['year']-2020)*12+new_old_df['month']
new_old_df.loc[new_old_df['月订单数量']==0, '月订单数量'] = np.nan
new_old_df = new_old_df.pivot(index='物料编号', columns='year_month', values='月订单数量')
# display(new_old_df)

# 考虑前12个月非0数，后12个月的非0数
total_nums = new_old_df.sum(axis=1).to_frame("nums").reset_index()
total_top12nozeros = np.sum(~new_old_df.iloc[:,:12].isnull(),axis=1).to_frame("top12nozeros").reset_index()
total_tail12nozeros = np.sum(~new_old_df.iloc[:,-12:].isnull(),axis=1).to_frame("tail12nozeros").reset_index()
tt = pd.merge(total_nums, total_top12nozeros, on='物料编号')
tt = pd.merge(tt, total_tail12nozeros, on='物料编号')

tt['配件生命周期'] = '生产期'
tt.loc[(tt['nums']>100) & (tt['top12nozeros']<5) & (tt['tail12nozeros']>5), '配件生命周期'] = '新配件'
tt.loc[(tt['nums']>100) & (tt['top12nozeros']>5) & (tt['tail12nozeros']<5), '配件生命周期'] = '老配件'
display(tt['配件生命周期'].value_counts())

生产期    20113
新配件      156
老配件       42
Name: 配件生命周期, dtype: int64

五、考虑需求为非0的频次，24个月若非0需求的频次小于等于4（待定），没有预测的必要；另外这一部分不需要预测的结合物料的价值和重要性可采用少量备货或不备货的策略

In [13]:
print(len(matrl_stats.loc[(matrl_stats['在产情况']=='在产') & (matrl_stats['单月0需求比例']<=0.8)]))
matrl_stats.loc[(matrl_stats['在产情况']=='在产') & (matrl_stats['单月0需求比例']<=0.8), '物料类别'].value_counts()

2452


一般结构件类       460
国标件          345
管和接头类        257
橡塑尼龙类        213
泵和阀类         194
钢丝绳类         132
传动件传动操纵类     126
油缸类          113
轴承类          109
发动机类          86
电器件类          64
润滑与密封类        52
辅材杂件类         36
液压件类          36
两室            28
吊臂(伸缩)        25
减速机           23
油品类           20
弹簧件类          20
起重设备类         20
小型结构件         16
标牌图册类         15
气动元件类         13
马达类           12
大型结构件类        10
滤芯类           10
吊臂(基本/顶节)      7
异型臂            4
两室配件           3
标准节            2
支腿(活动)         1
Name: 物料类别, dtype: int64

六、LLamasoft方案，结合需求量、频次（间隔）、方差波动分类

In [10]:
# 先按照LLamasoft的需求分类进行试分类，中间部分参数只能猜测

matrl_stats['分类'] = np.nan

# 24个月只有少量几个月有需求，这样的配件也无法预测，暂定为4(对应单月0需求比例大于0.8左右)
matrl_stats.loc[(matrl_stats['单月0需求比例']>0.8), '分类'] = 'Extremely Slow/Unpredictable'
matrl_stats.loc[(matrl_stats['单月0需求比例']<=0.8) & 
                (matrl_stats['单配件月非0需求标准差']>=10) & 
                (matrl_stats['单配件月需求最大值']>=10*matrl_stats['单配件月非0需求均值']), 
                '分类'] = 'Outlier'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']<1.32) &
                (matrl_stats['单配件月非0需求变异系数']<0.7), 
                '分类'] = 'Non-Intermittent Smooth'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']<1.32) &
                (matrl_stats['单配件月非0需求变异系数']>=0.7), 
                '分类'] = 'Non-Intermittent Erratic'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']>=1.32) &
                (matrl_stats['单配件月非0需求标准差']<4) & 
                (matrl_stats['单配件月非0需求变异系数']<0.7), 
                '分类'] = 'Intermittent Low Variable Slow'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']>=1.32) &
                (matrl_stats['单配件月非0需求标准差']<4) & 
                (matrl_stats['单配件月非0需求变异系数']>=0.7), 
                '分类'] = 'Intermittent Low Variable Lumpy'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']>=1.32) &
                (matrl_stats['单配件月非0需求标准差']>=4) & 
                (matrl_stats['单配件月非0需求变异系数']<0.7), 
                '分类'] = 'Intermittent High Variable Slow'
matrl_stats.loc[(matrl_stats['分类']!='Extremely Slow/Unpredictable') & 
                (matrl_stats['分类']!='Outlier') & 
                (matrl_stats['需求平均间隔']>=1.32) &
                (matrl_stats['单配件月非0需求标准差']>=4) & 
                (matrl_stats['单配件月非0需求变异系数']>=0.7), 
                '分类'] = 'Intermittent High Variable Lumpy'

In [39]:
display(matrl_stats['分类'].value_counts())
display(matrl_stats.loc[(matrl_stats['在产情况']=='在产') & (matrl_stats['单月0需求比例']<=0.8), '分类'].value_counts())

Extremely Slow/Unpredictable        15986
Intermittent Low Variable Slow       2039
Intermittent High Variable Lumpy      824
Intermittent Low Variable Lumpy       594
Non-Intermittent Smooth               337
Non-Intermittent Erratic              307
Intermittent High Variable Slow       211
Outlier                                13
Name: 分类, dtype: int64

Intermittent Low Variable Slow      1051
Intermittent High Variable Lumpy     542
Intermittent Low Variable Lumpy      324
Non-Intermittent Smooth              203
Non-Intermittent Erratic             181
Intermittent High Variable Slow      139
Outlier                               12
Name: 分类, dtype: int64

In [40]:
class_df = matrl_stats.loc[
    (matrl_stats['在产情况']=='在产') 
    & (matrl_stats['单月0需求比例']<=0.8)].groupby(['物料类别', '分类'], as_index=False).count()[['物料类别', '分类', '物料编号']]
class_df

,物料类别,分类,物料编号
0,一般结构件类,Intermittent High Variable Lumpy,65
1,一般结构件类,Intermittent High Variable Slow,16
2,一般结构件类,Intermittent Low Variable Lumpy,65
3,一般结构件类,Intermittent Low Variable Slow,263
4,一般结构件类,Non-Intermittent Erratic,25
5,一般结构件类,Non-Intermittent Smooth,26
6,两室,Intermittent Low Variable Lumpy,1
7,两室,Intermittent Low Variable Slow,22
8,两室,Non-Intermittent Erratic,1
9,两室,Non-Intermittent Smooth,4


七、考虑成本和价值、重要性等

八、其他方案，ABC、FSN等

九、聚类分析：须考虑对哪些物料和属性进行聚类，不同属性加权？

In [42]:
# 筛选参与聚类的数据和特征
# 不考虑在产情况为停产的配件，不考虑物料类别未知的配件
# 不考虑物料编号、计量单位、物料描述、吨位区间、设备类型等特征
cols = ['单配件月需求均值', '单配件月需求最大值', '单配件月需求标准差', '单配件月需求变异系数', '需求平均间隔',
#         '单配件月非0需求均值', '单配件月非0需求标准差', '单配件月非0需求变异系数', '单月0需求比例', 
        '周期（天）下计划后的交货周期', '物料单位成本']
data = matrl_stats[cols].copy()
data.fillna(0, inplace=True)
data

,单配件月需求均值,单配件月需求最大值,单配件月需求标准差,单配件月需求变异系数,需求平均间隔,周期（天）下计划后的交货周期,物料单位成本
0,0.041667,1.0,0.204124,4.898979,12.500000,未知,122.00
1,0.041667,1.0,0.204124,4.898979,12.500000,12,532.00
2,0.250000,3.0,0.675664,2.702656,5.000000,12,75.00
3,0.083333,1.0,0.282330,3.387958,8.333333,12,115.00
4,0.083333,1.0,0.282330,3.387958,8.333333,12,185.00
...,...,...,...,...,...,...,...
20306,0.083333,2.0,0.408248,4.898979,12.500000,8,0.00
20307,0.083333,1.0,0.282330,3.387958,8.333333,8,25.53
20308,0.208333,2.0,0.588230,2.823504,6.250000,8,24.25
20309,0.041667,1.0,0.204124,4.898979,12.500000,8,17.03


In [67]:
# 数据标准化
scaler  = RobustScaler().fit(data)
data_scale = scaler.transform(data)

In [74]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(data_scale)

In [75]:
pd.DataFrame(kmeans.labels_, columns=['labels'])['labels'].value_counts()

0     7812
10     493
14     467
17     171
16     107
9       35
15      25
11      10
8       10
13       9
7        4
19       3
6        3
3        2
4        1
18       1
12       1
2        1
5        1
1        1
Name: labels, dtype: int64

In [76]:
kmeans.score(data_scale)

-2241276.375972953